In [5]:
from pyspark.sql import functions as F, SparkSession
from pyspark.sql.types import IntegerType, LongType, DoubleType, StringType, DoubleType
from urllib.request import urlretrieve

In [2]:
spark = (
    SparkSession.builder.appName("Merchant Fraud Model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)

24/09/09 19:33:43 WARN Utils: Your hostname, DESKTOP-H6V94HM resolves to a loopback address: 127.0.1.1; using 192.168.0.204 instead (on interface wifi0)
24/09/09 19:33:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 19:33:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
url = "https://www.abs.gov.au/statistics/people/people-and-communities/snapshot-australia/2021/Snapshot%20of%20Australia%20data%20summary.xlsx"


# urlretrieve(url, "test.xlsx")

In [12]:
import requests

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [14]:
with open('data.csv', 'w') as f_out:
    f_out.write(requests.get(url).text)

In [ ]:
ad = spark.read.parquet()